<a href="https://colab.research.google.com/github/BeesieJF/BeesieJF/blob/main/CustomAssetAllocEnv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu==1.15.0 tensorflow==1.15.0 gym-anytrading gym stable_baselines

In [ ]:
import gym
import numpy as np
import pandas as pd
#from stable_baselines3 import MlpPolicy, CnnPolicy, CnnLstmPolicy
#from stable_baselines3 import PPO
#from stable_baselines3.common.env_util import make_vec_env
from matplotlib import pyplot as plt
from gym import spaces

In [ ]:
MAX_ACCOUNT_BALANCE = 200000
MAX_NUM_SHARES = 200000
MAX_SHARE_PRICE = 1000
MAX_OPEN_POSITIONS = 5
MAX_STEPS = 10000

INITIAL_ACCOUNT_BALANCE = 2000
print ('Initialization of variables done!')

Initialization of variables done!


In [ ]:
import random

class Env4AssetAlloc(gym.Env):
  """An environment for allocation of S&P500 Stocks in OpenAI gym"""
  metadata = {'render.modes': ['human']}
  
  def __init__(self, df):
    
    # Initialize an instance of the enviroment
    super(Env4AssetAlloc, self).__init__()
    self.df = df
    self.reward_range = (0, MAX_ACCOUNT_BALANCE) 
    
    # Actions are weights of stocks/assets - continuous
    self.action_space = spaces.Box(low=np.array([0, 0]), high=np.array([3, 1]), dtype=np.float16)
    
    # Price obs space contains the OHLCV values for the last five prices
    self.observation_space = spaces.Box(low=0, high=1, shape=(6, 6), dtype=np.float16)

    return #self.action_space, self.observation_space
    
  def reset(self):
    
    # Reset the state and environment variables to initial values
    self.balance = INITIAL_ACCOUNT_BALANCE
    self.net_worth = INITIAL_ACCOUNT_BALANCE
    self.max_net_worth = INITIAL_ACCOUNT_BALANCE
    self.assets_held = 0
    self.cost_basis = 0
    self.total_assets_sold = 0
    self.total_held_assets_value = 0
 
    # Set the current step to a random point within the data frame
    self.current_step = random.randint(0, len(self.df.loc[:, 'Open'].values) - 3)

    print ('Present step is', self.current_step)
print ('Initialization of environment and Reset functions done!')

Initialization of environment and Reset functions done!


In [ ]:
def _next_observation(self):

    # Get the data points for the last 5 days and scale to between 0-1
      frame = np.array([self.df.loc[self.current_step: self.current_step + 5, 'Open'].values / MAX_SHARE_PRICE, 
                      self.df.loc[self.current_step: self.current_step + 5, 'High'].values / MAX_SHARE_PRICE,
                      self.df.loc[self.current_step: self.current_step + 5, 'Low'].values / MAX_SHARE_PRICE,
                      self.df.loc[self.current_step: self.current_step + 5, 'Close'].values / MAX_SHARE_PRICE,
                      self.df.loc[self.current_step: self.current_step + 5, 'Volume'].values / MAX_NUM_SHARES,])
  
    # Append additional data and scale each value to between 0-1
      obs = np.append(frame, [[self.balance / MAX_ACCOUNT_BALANCE, 
                             self.max_net_worth / MAX_ACCOUNT_BALANCE,
                             self.shares_held / MAX_NUM_SHARES,
                             self.cost_basis / MAX_SHARE_PRICE,
                             self.total_shares_sold / MAX_NUM_SHARES,
                             self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),]], axis=0)
      print (obs)

def  step(self, action):
    
    # Execute one time step within the environment
      self._take_action(action)
      self.current_step += 1
      if self.current_step > len(self.df.loc[:, 'Open'].values) - 6:
        self.current_step = 0
      delay_modifier = (self.current_step / MAX_STEPS)
  
      reward = self.balance * delay_modifier
      done = self.net_worth <= 0
      obs = self._next_observation()
      return obs, reward, done, {}

print ('Next Observation and Step functions done!')

Next Observation and Step functions done!


In [ ]:
def _take_action(self, action):
  
    # Set the current price to a random price within the time step
    current_price = random.uniform(
    self.df.loc[self.current_step, "Open"],
    self.df.loc[self.current_step, "Close"])
    action_type = action[0]
    amount = action[1]
    if action_type < 1:
   
    # Calaculate % amount of balance to buy (of shares)
      total_possible = self.balance / current_price
      shares_bought = total_possible * amount
      prev_cost = self.cost_basis * self.shares_held
      additional_cost = shares_bought * current_price
      self.balance -= additional_cost
      self.cost_basis = (prev_cost + additional_cost) / (self.shares_held + shares_bought)
      self.shares_held += shares_bought

    elif action_type < 2:
      
    # Sell % amount of shares held
      shares_sold = self.shares_held * amount
      self.balance += shares_sold * current_price
      self.shares_held -= shares_sold
      self.total_shares_sold += shares_sold
      self.total_sales_value += shares_sold * current_price
      self.netWorth = self.balance + self.shares_held * current_price
    if self.net_worth > self.max_net_worth:
      self.max_net_worth = self.net_worth
    if self.shares_held == 0:
      self.cost_basis = 0

def render (self, mode='human', close=False):
        # Render the environment to the screen
      profit = self.net_worth - INITIAL_ACCOUNT_BALANCE
      print(f'Step: {self.current_step}')
      print(f'Balance: {self.balance}')
      print(f'Shares held: {self.shares_held} (Total sold: {self.total_shares_sold})')
      print(f'Avg cost for held shares: {self.cost_basis} (Total sales value: {self.total_sales_value})')
      print(f'Net worth: {self.net_worth} (Max net worth: {self.max_net_worth})')
      print(f'Profit: {profit}')

print ('Take action and Enviroment Render functions done!')

Take action and Enviroment Render functions done!


In [ ]:
class Env4StockTrading(gym.Env):
  """A trading environment for S&P500 Stocks in OpenAI gym"""
  metadata = {'render.modes': ['human']}
  
  def __init__(self, df):
    
    # Initialize an instance of the trading enviroment
    super(Env4StockTrading, self).__init__()
    self.df = df
    self.reward_range = (0, MAX_ACCOUNT_BALANCE) 
    
    # Actions are Buy, Sell, Hold
    self.action_space = spaces.Box(low=np.array([0, 0]), high=np.array([3, 1]), dtype=np.float16)
    
    # Price obs space contains the OHLCV values for the last five prices
    self.observation_space = spaces.Box(low=0, high=1, shape=(6, 6), dtype=np.float16)

    return self.action_space, self.observation_space
    
  def reset(self):
    
    # Reset the state and environment variables to initial values
    self.balance = INITIAL_ACCOUNT_BALANCE
    self.net_worth = INITIAL_ACCOUNT_BALANCE
    self.max_net_worth = INITIAL_ACCOUNT_BALANCE
    self.shares_held = 0
    self.cost_basis = 0
    self.total_shares_sold = 0
    self.total_sales_value = 0
 
    # Set the current step to a random point within the data frame
    self.current_step = random.randint(0, len(self.df.loc[:, 'Open'].values) - 3)

    print ('Present step is', self.current_step)
    return self._next_observation()

  def _next_observation(self):

    # Get the data points for the last 5 days and scale to between 0-1
    frame = np.array([
    self.df.loc[self.current_step: self.current_step +
                5, 'Open'].values / MAX_SHARE_PRICE,
    self.df.loc[self.current_step: self.current_step +
                5, 'High'].values / MAX_SHARE_PRICE,
    self.df.loc[self.current_step: self.current_step +
                5, 'Low'].values / MAX_SHARE_PRICE,
    self.df.loc[self.current_step: self.current_step +
                5, 'Close'].values / MAX_SHARE_PRICE,
    self.df.loc[self.current_step: self.current_step +
                5, 'Volume'].values / MAX_NUM_SHARES,
   ])
  
    # Append additional data and scale each value to between 0-1
    obs = np.append(frame, [[
    self.balance / MAX_ACCOUNT_BALANCE,
    self.max_net_worth / MAX_ACCOUNT_BALANCE,
    self.shares_held / MAX_NUM_SHARES,
    self.cost_basis / MAX_SHARE_PRICE,
    self.total_shares_sold / MAX_NUM_SHARES,
    self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
  ]], axis=0)
    print (obs)

  def  step(self, action):
    
    # Execute one time step within the environment
    self._take_action(action)
    self.current_step += 1
    if self.current_step > len(self.df.loc[:, 'Open'].values) - 6:
      self.current_step = 0
    delay_modifier = (self.current_step / MAX_STEPS)
  
    reward = self.balance * delay_modifier
    done = self.net_worth <= 0
    obs = self._next_observation()
    return obs, reward, done, {}

In [ ]:
  def _take_action(self, action):
  
    # Set the current price to a random price within the time step
    current_price = random.uniform(
    self.df.loc[self.current_step, "Open"],
    self.df.loc[self.current_step, "Close"])
    action_type = action[0]
    amount = action[1]
    if action_type < 1:
   
    # Calaculate % amount of balance to buy (of shares)
      total_possible = self.balance / current_price
      shares_bought = total_possible * amount
      prev_cost = self.cost_basis * self.shares_held
      additional_cost = shares_bought * current_price
      self.balance -= additional_cost
      self.cost_basis = (prev_cost + additional_cost) / (self.shares_held + shares_bought)
      self.shares_held += shares_bought

    elif action_type < 2:

    # Sell % amount of shares held
      shares_sold = self.shares_held * amount
      self.balance += shares_sold * current_price
      self.shares_held -= shares_sold
      self.total_shares_sold += shares_sold
      self.total_sales_value += shares_sold * current_price
      self.netWorth = self.balance + self.shares_held * current_price
    if self.net_worth > self.max_net_worth:
      self.max_net_worth = self.net_worth
    if self.shares_held == 0:
      self.cost_basis = 0

def render (self, mode='human', close=False):
  # mode='human', close=False):
  # Render the environment to the screen
    profit = self.net_worth - INITIAL_ACCOUNT_BALANCE
    print(f'Step: {self.current_step}')
    print(f'Balance: {self.balance}')
    print(f'Shares held: {self.shares_held} (Total sold: {self.total_shares_sold})')
    print(f'Avg cost for held shares: {self.cost_basis} (Total sales value: {self.total_sales_value})')
    print(f'Net worth: {self.net_worth} (Max net worth: {self.max_net_worth})')
    print(f'Profit: {profit}')

In [ ]:
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

#import gym
import json
import datetime as dt
from stable_baselines import PPO2
from stable_baselines.common.vec_env import DummyVecEnv
#from stable_baselines.common import make_vec_env
from stable_baselines.common.policies import MlpPolicy
from matplotlib import pyplot as plt
from gym import envs

import pandas as pd
from gym_anytrading.datasets import STOCKS_GOOGL

# Initialize a vectorized environment
df = STOCKS_GOOGL
env = DummyVecEnv([lambda: Env4StockTrading(df)])
#env = DummyVecEnv(Env4StockTrading(df))

model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=10000)
model.save("ppo_StockTrade")

obs = env.reset()
#for i in range(2000):
  #action, _states = model.predict(obs)
  #obs, rewards, done, info = env.step(action)
  #env.render()